In [ ]:
! pip install yt-dlp


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/3.3 MB ? eta -:--:--
   ---------------------------------------- 3.3/3.3 MB 16.6 MB/s  0:00:00


In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import subprocess
import threading
import os
import sys

# -----------------------------
# CONFIG
# -----------------------------
DEFAULT_URL = "https://youtube.com/playlist?list=PLtH28TLJFEJaL125qW_6HN5w8M5lZE32u"
DEFAULT_DIR = os.path.join(os.path.expanduser("~"), "Downloads", "YT_Playlist")

YTDLP_CMD = "yt-dlp"
if sys.platform.startswith("win") and os.path.exists("yt-dlp.exe"):
    YTDLP_CMD = "yt-dlp.exe"

# -----------------------------
# GUI APP
# -----------------------------
class YTDownloader(tk.Tk):
    def __init__(self):
        super().__init__()

        self.title("YouTube Playlist Downloader")
        self.geometry("720x520")
        self.resizable(False, False)

        self.url_var = tk.StringVar(value=DEFAULT_URL)
        self.dir_var = tk.StringVar(value=DEFAULT_DIR)
        self.mode_var = tk.StringVar(value="video")
        self.quality_var = tk.StringVar(value="best")

        self.create_widgets()

    def create_widgets(self):
        pad = 8

        tk.Label(self, text="YouTube URL (Video or Playlist)").pack(anchor="w", padx=pad, pady=(10, 0))
        tk.Entry(self, textvariable=self.url_var, width=90).pack(padx=pad)

        tk.Label(self, text="Download Folder").pack(anchor="w", padx=pad, pady=(10, 0))
        frame = tk.Frame(self)
        frame.pack(fill="x", padx=pad)
        tk.Entry(frame, textvariable=self.dir_var, width=70).pack(side="left", expand=True)
        tk.Button(frame, text="Browse", command=self.browse_dir).pack(side="right")

        options = tk.Frame(self)
        options.pack(fill="x", padx=pad, pady=10)

        tk.Label(options, text="Mode:").grid(row=0, column=0, sticky="w")
        ttk.Combobox(options, values=["video", "audio (mp3)"], textvariable=self.mode_var, width=15).grid(row=0, column=1)

        tk.Label(options, text="Quality:").grid(row=0, column=2, padx=(20, 0))
        ttk.Combobox(options, values=["best", "720p", "480p"], textvariable=self.quality_var, width=10).grid(row=0, column=3)

        tk.Button(self, text="START DOWNLOAD", bg="#1abc9c", fg="white",
                  font=("Segoe UI", 11, "bold"),
                  command=self.start_download).pack(pady=10)

        self.log = tk.Text(self, height=16, bg="#111", fg="#0f0", font=("Consolas", 9))
        self.log.pack(fill="both", expand=True, padx=pad, pady=(0, 10))

    def browse_dir(self):
        folder = filedialog.askdirectory()
        if folder:
            self.dir_var.set(folder)

    def log_write(self, text):
        self.log.insert("end", text + "\n")
        self.log.see("end")

    def start_download(self):
        threading.Thread(target=self.download, daemon=True).start()

    def download(self):
        url = self.url_var.get().strip()
        folder = self.dir_var.get().strip()

        if not url:
            messagebox.showerror("Error", "Please enter a YouTube URL")
            return

        os.makedirs(folder, exist_ok=True)

        cmd = [YTDLP_CMD, "--yes-playlist", "-o", f"{folder}/%(playlist_index)s - %(title)s.%(ext)s"]

        if "audio" in self.mode_var.get():
            cmd += ["-x", "--audio-format", "mp3"]

        q = self.quality_var.get()
        if q == "720p":
            cmd += ["-f", "bestvideo[height<=720]+bestaudio/best"]
        elif q == "480p":
            cmd += ["-f", "bestvideo[height<=480]+bestaudio/best"]

        cmd.append(url)

        self.log_write("COMMAND:")
        self.log_write(" ".join(cmd))
        self.log_write("=" * 60)

        try:
            process = subprocess.Popen(
                cmd,
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                text=True
            )

            for line in process.stdout:
                self.log_write(line.strip())

            self.log_write("\nDownload finished ✔")

        except Exception as e:
            self.log_write(f"ERROR: {e}")

# -----------------------------
# RUN
# -----------------------------
if __name__ == "__main__":
    app = YTDownloader()
    app.mainloop()
